In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import operator

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

transaction_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
transaction_df.head()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Function to get common articles from Users' transaction histories
def get_common_articles(userA,userB):
    userA_List = transaction_df[transaction_df.customer_id == userA]
    userB_List = transaction_df[transaction_df.customer_id == userB]
    return [article for article in userA_List['article_id'] if article in userB_List['article_id']]

get_common_articles('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318','00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# articles_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv",header=0)
customers_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv", header=0)
# transactions_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv",header=0)
# # 创建评分系统：根据所有用户的购买记录统计某一段时间内的单个 item 的 popularity
# print(transactions.columns[0])
# transactions['t_dat'].unique()
# """
# ['customer_id', 'FN', 'Active', 'club_member_status',
#        'fashion_news_frequency', 'age', 'postal_code']
# """
# club_member_status: ACTIVE PRE-CREATE LEFT CLUB
# fashion_news_frequency: NONE Regularly Monthly None
club_member_status_dict = {
    'ACTIVE': 3,
    'PRE-CREATE': 2,
    'LEFT CLUB': 1
}
fashion_news_frequency_dict = {
    'NONE': 1,
    'Regularly': 2,
    'Monthly': 3,
    'None': 1
}
# print(customers_df['club_member_status'].unique())
customers_ids = []
customers_points = []
transactions_df = transaction_df

for i in range(len(customers_df['club_member_status'])):
    status1 = customers_df['club_member_status'][i]
    if status1 in club_member_status_dict:
        points = club_member_status_dict[status1]
    else:
        points = 1
    status2 = customers_df['fashion_news_frequency'][i]
    if status2 in fashion_news_frequency_dict:
        points += fashion_news_frequency_dict[status2]
    else:
        points += 1
    customers_ids.append(customers_df['customer_id'][i])
    customers_points.append(points)

temp_df = pd.DataFrame(list(zip(customers_ids, customers_points)), columns=['customer_id', 'weight'])
# transactions_df = transactions_df.merge(temp_df, on='customer_id')

# transactions_df.to_csv('transactions_weighted.csv')
transactions_weighted = transactions_df.merge(temp_df, on='customer_id')

In [ ]:
# Find the same articles in users' transaction_rating table;
def get_reviews(userA,userB):
    common_articles = get_common_articles(userA,userB)
    return [(transactions_weighted[userA][article], transactions_weighted[userB][article]) for article in common_articles]

In [ ]:
# Function to get Euclidean Distance b/w 2 points
import math
def euclidean_distance(points):
    squared_diffs = [(point[0] - point[1]) ** 2 for point in points]
    summed_squared_diffs = sum(squared_diffs)
    distance = math.sqrt(summed_squared_diffs)
    return distance

In [ ]:
# Function to  calculate similarity more similar less the distance and vice versa
# Added 1 for if highly similar can make the distance zero and give NotDefined Error
def similarity(reviews):
    return 1/ (1 + euclidean_distance(reviews))

In [ ]:
# get similarity via two users' transactions behavior;
def get_transaction_similarity(userA, userB):
    reviews = get_reviews(userA,userB)
    return similarity(reviews)

In [ ]:
import operator
# Function to give recommendation to users based on their reviews.
def recommend_articles(user, num_suggestions):
    review_data = transaction_df[transaction_df.customer_id == user]
    similarity_scores = [(get_transaction_similarity(user, other), other) for other in review_data if other != user]

    similarity_scores.sort()
    similarity_scores.reverse()
    similarity_scores = similarity_scores[0:num_suggestions]


    recommendations = {}

    for similarity, other in similarity_scores:
        reviewed = review_data[other]

        for movie in reviewed:
            if movie not in review_data[user]:
                weight = similarity * reviewed[movie]

                if movie in recommendations:
                    sim, weights = recommendations[movie]
                    recommendations[movie] = (sim + similarity, weights + [weight])
                else:
                    recommendations[movie] = (similarity, [weight])

    for recommendation in recommendations:
        similarity, movie = recommendations[recommendation]
        recommendations[recommendation] = sum(movie) / similarity

    sorted_recommendations = sorted(recommendations.items(), key=operator.itemgetter(1), reverse=True)
    print("get recommendations")
    return sorted_recommendations

In [ ]:
recommend_movies('00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280',4)


In [ ]:
predictions = []
customer_ids = []
customer = customers_df['customer_id']
for id in customer:
    try:
        recommendations = recommend_articles(id, 12)
    except:
        recommendations = ['0']
    predictions.append(recommendations)
    customer_ids.append(id)
results_df = pd.DataFrame(list(zip(customer_ids, predictions)), columns=['customer_id', 'prediction'])
results_df.to_csv('prediction_results.csv')

In [ ]:
# Find Most Popular Articles;
transaction_df = transaction_df[transaction_df.t_dat > '2020-09-07']

# What items do each customers buy the most
cp = transaction_df.groupby(["customer_id", "article_id"])[["article_id"]].count()
cp.columns = ["purchase_count"]
cp = cp.reset_index()

cp = cp.sort_values(["customer_id", "purchase_count"], ascending=False)
cp = cp.groupby("customer_id").head(10)
cp = cp.groupby("customer_id")["article_id"].apply(list)

transaction_df['article_id'].value_counts()[:12]

pp = list(transaction_df['article_id'].value_counts().index[:12])

In [ ]:
# Submission
pp = ['0909370001', '0924243001', '0918522001', '0865799006', '0751471001', '0448509014', '0762846027', '0918292001', '0923758001', '0924243002', '0915529003', '0850917001']
submission_df = pd.read_csv('data/sample_submission.csv')
submission_df.head()

submission_df["prediction"] = submission_df["customer_id"].map(cp)
submission_df["prediction"] = submission_df["prediction"].apply(lambda x: x if isinstance(x, list) else [])

submission_df["prediction"] = submission_df["prediction"].apply(lambda x: x[:12] + pp[:12-len(x)])
submission_df["prediction"] = submission_df["prediction"].apply(lambda x: " ".join(x))
submission_df.to_csv('simple_submission.csv', index=False)